In [22]:
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopy
from pandarallel import pandarallel

In [5]:
df = pd.read_csv('Private Residential Property Transactions with MRT, Bus, Taxi Data.csv')
df.drop(columns=df.columns[0], axis=1,  inplace=True)

In [6]:
df.head()

,street,x,project,y,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,...,nearest_mrt_walking_distance,nearest_stop_num,nearest_stop_name,distance_from_nearest_stop,nearest_stop_lat_long,nearest_taxi_stand,distance_from_nearest_taxi_stand,nearest_taxi_stand_lat_long,nearest_taxi_stand_walking_time,nearest_taxi_stand_walking_distance
0,ZEHNDER ROAD,22734.9253,LANDED HOUSING DEVELOPMENT,29502.42071,RCR,524.3,-,1,220,3,...,4.0,15091,Opp Natl Leadership Inst,0.201405,"(1.28290732015735, 103.78781105381566)",Pasir Panjang Rd outside Haw Par Villa CCL Sta...,0.481392,"(1.283306206, 103.78169)",NaN,NaN
1,ZEHNDER ROAD,22734.9253,LANDED HOUSING DEVELOPMENT,29502.42071,RCR,308.0,-,1,918,3,...,4.0,15091,Opp Natl Leadership Inst,0.201405,"(1.28290732015735, 103.78781105381566)",Pasir Panjang Rd outside Haw Par Villa CCL Sta...,0.481392,"(1.283306206, 103.78169)",NaN,NaN
2,ZEHNDER ROAD,22734.9253,LANDED HOUSING DEVELOPMENT,29502.42071,RCR,314.0,-,1,618,3,...,4.0,15091,Opp Natl Leadership Inst,0.201405,"(1.28290732015735, 103.78781105381566)",Pasir Panjang Rd outside Haw Par Villa CCL Sta...,0.481392,"(1.283306206, 103.78169)",NaN,NaN
3,ZEHNDER ROAD,22734.9253,LANDED HOUSING DEVELOPMENT,29502.42071,RCR,308.0,-,1,921,3,...,4.0,15091,Opp Natl Leadership Inst,0.201405,"(1.28290732015735, 103.78781105381566)",Pasir Panjang Rd outside Haw Par Villa CCL Sta...,0.481392,"(1.283306206, 103.78169)",NaN,NaN
4,NEO PEE TECK LANE,20914.1989,LANDED HOUSING DEVELOPMENT,30589.10708,RCR,159.3,-,1,320,3,...,2.0,16089,Aft Clementi Rd,0.117683,"(1.29201138346787, 103.76908420716119)",Pasir Panjang Rd outside Haw Par Villa,1.636447,"(1.28376027, 103.7812063)",NaN,NaN


In [18]:
# Create dictionary of postal districts
# Obtained from https://www.creatifwerks.com/singapore-district-code-finder/
districts = {}
districts[1] = ["Raffles Place", "Cecil", "Marina", "People's Park"]
districts[2] = ["Anson", "Tanjong Pagar"]
districts[3] = ["Queenstown", "Tiong Bahru"]
districts[4] = ["Telok Blangah", "Harbourfront"]
districts[5] = ["Pasir Panjang", "Hong Leong Garden", "Clementi New Town"]
districts[6] = ["High Street", "Beach Road"]
districts[7] = ["Middle Road", "Golden Mile"]

districts_by_postal_code = {}
districts_by_postal_code[1] = ["01", "02", "03", "04", "05", "06"]
districts_by_postal_code[2] = ["07", "08"]
districts_by_postal_code[3] = ["14", "15", "16"]
districts_by_postal_code[4] = ["09", "10"]
districts_by_postal_code[5] = ["11", "12", "13"]
districts_by_postal_code[6] = ["17"]
districts_by_postal_code[7] = ["18", "19"]

In [7]:
# Neighbourhood safety ranking obtained from https://www.valuechampion.sg/2018/07/5-safest-neighbourhoods-singapore
npc_safety = {"Bukit Timah": 1, "Bishan": 2, "Kampong Java": 3, "Marine Parade": 4, "Sembawang": 5, "Marina Bay": 6, "Serangoon": 7, 
              "Changi": 8, "Bedok South": 9, "Queenstown": 10, "Bukit Merah West": 11, "Punggol": 12, "Jurong East": 13, 
              "Ang Mo Kio South": 14, "Orchard": 15, "Pasir Ris": 16, "Bukit Merah East": 17, "Rochor": 18, "Clementi": 19, 
              "Yishun South": 20, "Toa Payoh": 21, "Ang Mo Kio North": 22, "Bukit Panjang": 23, "Woodlands West": 24, "Bukit Batok": 25, 
              "Sengkang": 26, "Bedok North": 27,  "Hougang": 28, "Geylang": 29, "Choa Chu Kang": 30, "Woodlands East": 31, "Tampines": 32, 
              "Jurong West": 33, "Nanyang": 34, "Yishun North": 35}

In [10]:
# NPC coordinates obtained from https://www.onemap.gov.sg/main/v2/
npc_coordinates = {"Bukit Timah": (1.3227233, 103.8124098), "Bishan": (1.3578060, 103.8477093), "Kampong Java": (1.3120434, 103.8464821), 
                   "Marine Parade": (1.3062879, 103.9145216), "Sembawang": (1.4452778, 103.8196465), "Marina Bay": (1.2747355, 103.8489943), 
                   "Serangoon": (1.3519722, 103.8706143), "Changi": (1.3445539, 103.9548303), "Bedok South": (1.3237020, 103.9198174), 
                   "Queenstown": (1.2884116, 103.8027820), "Bukit Merah West": (1.2856083, 103.8234001), "Punggol": (1.3978397, 103.9138862), 
                   "Jurong East": (1.3401265, 103.7372282), "Ang Mo Kio South": (1.3699100, 103.8516535), "Orchard": (1.2989882, 103.8398419), 
                   "Pasir Ris": (1.3683491, 103.9595951), "Bukit Merah East": (1.2784826, 103.8394162), "Rochor": (1.3076488, 103.8542707), 
                   "Clementi": (1.3171721, 103.7665558), "Yishun South": (1.4153202, 103.8348964), "Toa Payoh": (1.3346258, 103.8504670), 
                   "Ang Mo Kio North": (1.3850097, 103.8450951), "Bukit Panjang": (1.3868890, 103.7717196), "Woodlands West": (1.4334971, 103.7788583), 
                   "Bukit Batok": (1.3491092, 103.7585217), "Sengkang": (1.3927856, 103.8939061), "Bedok North": (1.3327094, 103.9370616),  
                   "Hougang": (1.3743099, 103.8849844), "Geylang": (1.3109665, 103.8861369), "Choa Chu Kang": (1.3948056, 103.7449713), 
                   "Woodlands East": (1.4388861, 103.8023648), "Tampines": (1.3462476, 103.9387018), "Jurong West": (1.3493325, 103.7144562), 
                   "Nanyang": (1.3508868, 103.7021111), "Yishun North": (1.4293427, 103.8402818)}

In [14]:
# Convert dict to df
npc_safety_df = pd.DataFrame(list(npc_safety.items()), columns=["Key", "Value"])
npc_safety_df

,Key,Value
0,Bukit Timah,1
1,Bishan,2
2,Kampong Java,3
3,Marine Parade,4
4,Sembawang,5
5,Marina Bay,6
6,Serangoon,7
7,Changi,8
8,Bedok South,9
9,Queenstown,10


In [15]:
# Convert dict to df
npc_coordinates_df = pd.DataFrame(list(npc_coordinates.items()), columns=["Key", "Value"])
npc_coordinates_df

,Key,Value
0,Bukit Timah,"(1.3227233, 103.8124098)"
1,Bishan,"(1.357806, 103.8477093)"
2,Kampong Java,"(1.3120434, 103.8464821)"
3,Marine Parade,"(1.3062879, 103.9145216)"
4,Sembawang,"(1.4452778, 103.8196465)"
5,Marina Bay,"(1.2747355, 103.8489943)"
6,Serangoon,"(1.3519722, 103.8706143)"
7,Changi,"(1.3445539, 103.9548303)"
8,Bedok South,"(1.323702, 103.9198174)"
9,Queenstown,"(1.2884116, 103.802782)"


In [17]:
# Merge dataframes
npc_df = pd.merge(npc_safety_df, npc_coordinates_df, on="Key")
npc_df

,Key,Value_x,Value_y
0,Bukit Timah,1,"(1.3227233, 103.8124098)"
1,Bishan,2,"(1.357806, 103.8477093)"
2,Kampong Java,3,"(1.3120434, 103.8464821)"
3,Marine Parade,4,"(1.3062879, 103.9145216)"
4,Sembawang,5,"(1.4452778, 103.8196465)"
5,Marina Bay,6,"(1.2747355, 103.8489943)"
6,Serangoon,7,"(1.3519722, 103.8706143)"
7,Changi,8,"(1.3445539, 103.9548303)"
8,Bedok South,9,"(1.323702, 103.9198174)"
9,Queenstown,10,"(1.2884116, 103.802782)"


In [19]:
# Rename columns
npc_df.columns = ["NPC", "Crime Rate Ranking", "Coordinates"]
npc_df.head()

,NPC,Crime Rate Ranking,Coordinates
0,Bukit Timah,1,"(1.3227233, 103.8124098)"
1,Bishan,2,"(1.357806, 103.8477093)"
2,Kampong Java,3,"(1.3120434, 103.8464821)"
3,Marine Parade,4,"(1.3062879, 103.9145216)"
4,Sembawang,5,"(1.4452778, 103.8196465)"


In [21]:
npc_df[["lat", "long"]] = npc_df["Coordinates"].apply(pd.Series)
npc_df.head()

,NPC,Crime Rate Ranking,Coordinates,lat,long
0,Bukit Timah,1,"(1.3227233, 103.8124098)",1.322723,103.812410
1,Bishan,2,"(1.357806, 103.8477093)",1.357806,103.847709
2,Kampong Java,3,"(1.3120434, 103.8464821)",1.312043,103.846482
3,Marine Parade,4,"(1.3062879, 103.9145216)",1.306288,103.914522
4,Sembawang,5,"(1.4452778, 103.8196465)",1.445278,103.819647


In [26]:
pandarallel.initialize(progress_bar=True)

def crime_rate(row, ranks=npc_df):
    from geopy.distance import geodesic

    min_distance, nearest_npc, rank = 10000, "", ""
    lat, long = row['lat_long'].split(',')
    
    for npc, ranking, latitude, longitude in zip(ranks['NPC'], ranks['Crime Rate Ranking'], ranks['lat'], ranks['long']):
        distance = geodesic((float(lat[1:]), float(long[1:-1])), (latitude, longitude)).km
        if distance < min_distance:
            min_distance = distance
            nearest_npc = npc
            rank = ranking
    return nearest_npc, rank

df['npc'], df['crime_rate_ranking'] = zip(*df.parallel_apply(crime_rate, axis=1))

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [27]:
df.head()

,street,x,project,y,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,...,nearest_stop_name,distance_from_nearest_stop,nearest_stop_lat_long,nearest_taxi_stand,distance_from_nearest_taxi_stand,nearest_taxi_stand_lat_long,nearest_taxi_stand_walking_time,nearest_taxi_stand_walking_distance,npc,crime_rate_ranking
0,ZEHNDER ROAD,22734.9253,LANDED HOUSING DEVELOPMENT,29502.42071,RCR,524.3,-,1,220,3,...,Opp Natl Leadership Inst,0.201405,"(1.28290732015735, 103.78781105381566)",Pasir Panjang Rd outside Haw Par Villa CCL Sta...,0.481392,"(1.283306206, 103.78169)",NaN,NaN,Queenstown,10
1,ZEHNDER ROAD,22734.9253,LANDED HOUSING DEVELOPMENT,29502.42071,RCR,308.0,-,1,918,3,...,Opp Natl Leadership Inst,0.201405,"(1.28290732015735, 103.78781105381566)",Pasir Panjang Rd outside Haw Par Villa CCL Sta...,0.481392,"(1.283306206, 103.78169)",NaN,NaN,Queenstown,10
2,ZEHNDER ROAD,22734.9253,LANDED HOUSING DEVELOPMENT,29502.42071,RCR,314.0,-,1,618,3,...,Opp Natl Leadership Inst,0.201405,"(1.28290732015735, 103.78781105381566)",Pasir Panjang Rd outside Haw Par Villa CCL Sta...,0.481392,"(1.283306206, 103.78169)",NaN,NaN,Queenstown,10
3,ZEHNDER ROAD,22734.9253,LANDED HOUSING DEVELOPMENT,29502.42071,RCR,308.0,-,1,921,3,...,Opp Natl Leadership Inst,0.201405,"(1.28290732015735, 103.78781105381566)",Pasir Panjang Rd outside Haw Par Villa CCL Sta...,0.481392,"(1.283306206, 103.78169)",NaN,NaN,Queenstown,10
4,NEO PEE TECK LANE,20914.1989,LANDED HOUSING DEVELOPMENT,30589.10708,RCR,159.3,-,1,320,3,...,Aft Clementi Rd,0.117683,"(1.29201138346787, 103.76908420716119)",Pasir Panjang Rd outside Haw Par Villa,1.636447,"(1.28376027, 103.7812063)",NaN,NaN,Clementi,19


In [30]:
df.to_csv("Transactions with MRT, Bus, Taxi, Crime.csv", index=False)